---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    train.csv - the training set (all tickets issued 2004-2011)
    test.csv - the test set (all tickets issued 2012-2016)
    addresses.csv & latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32
       
### Hints

* Make sure your code is working before submitting it to the autograder.

* Print out your result to see whether there is anything weird (e.g., all probabilities are the same).

* Generally the total runtime should be less than 10 mins. You should NOT use Neural Network related classifiers (e.g., MLPClassifier) in this question. 

* Try to avoid global variables. If you have other functions besides blight_model, you should move those functions inside the scope of blight_model.

* Refer to the pinned threads in Week 4's discussion forum when there is something you could not figure it out.

In [97]:
import pandas as pd
import numpy as np

def blight_model():
    
    # Minimalistic version. Use RF, remove str predictor cols
    
    # Your code here
    import pandas as pd
    import numpy as np
    
    from sklearn.model_selection import train_test_split
    
    
    traindata = pd.read_csv("train.csv", encoding = 'ISO-8859-1')
    testdata = pd.read_csv('test.csv')
    
    addresses =  pd.read_csv('addresses.csv')
    coords = pd.read_csv('latlons.csv')
    
    #print(addresses.head())
    #print("\n")
    #print(coords.head())
    
    
    # Filter null compliance rows from train (test does not have these) -- these could be ignored
    # keep only 0 and 1
    traindata = traindata[(traindata["compliance"] == 0) | (traindata["compliance"] == 1)]
    # enclose in parantheses...
    
    #print(traindata.head(15))
    
    # predictor columns (that can be find both from train and test)
    predictor_cols = ["ticket_id", "agency_name", "inspector_name", "violator_name", 
                      "violation_street_number", "violation_street_name", "violation_zip_code",
                     "mailing_address_str_number", "mailing_address_str_name", "city", "state", "zip_code",
                     "non_us_str_code", "country",
                     "ticket_issued_date", "hearing_date", 
                     "violation_code", "violation_description", "disposition",
                     "fine_amount", "admin_fee", "state_fee", "late_fee","discount_amount","clean_up_cost",
                     "judgment_amount", "grafitti_status"]
    
    #X_train = traindata.loc[:, traindata.columns != 'compliance']
    X_test = testdata
    
    y_train = traindata[["compliance"]]
    X_train = traindata[predictor_cols]
    #y_test 
    
    #print("1: ", X_train.shape)
    
    
    # Join address
    #X_train = X_train.merge(addresses, how="left", on="ticket_id")
    #X_test = X_test.merge(addresses, how="left", on="ticket_id")
    
    #print("2: ", X_train.shape)
    
    # Join coordinates and addresses
    # This multiplies the nbr of rows...
    #X_train = X_train.merge(addresses, how="left", on="address")
    #X_test = X_test.merge(addresses, how="left", on="address")
    
    
    addr_coords = addresses.set_index("address").join(coords.set_index("address"), how="left")
    
    # Näin voi kanssa kiertää... mutta meneekö suoraan set_indexillä kivemmin?
    # X_train= X_train.groupby(['ticket_id_x'])['address'].first().reset_index()
    # X_test = X_test.groupby(['ticket_id_x'])['address'].first().reset_index()
    
    X_train = X_train.set_index("ticket_id").join(addr_coords.set_index("ticket_id"), how="left")
    X_test = X_test.set_index("ticket_id").join(addr_coords.set_index("ticket_id"), how="left")
    
    #print("3: ", X_train.shape)
    
    #X_train.drop("ticket_id_y", axis=1, inplace=True)
    #X_test.drop("ticket_id_y", axis=1, inplace=True)
    
    #print("4: ", X_train.shape)
    
    # To begin with, remove string columns (or create one hot etc)
    str_predictor_cols = ['violator_name', 'country', 'city',
            'inspector_name', 'violation_street_number', 'violation_street_name',
            'violation_zip_code', 'violation_description','mailing_address_str_number', 
            'mailing_address_str_name', 'non_us_str_code', 'agency_name', 'state', 'disposition',
            'ticket_issued_date', 'hearing_date', 'grafitti_status', 'violation_code'
                ]
    
    # Removed zip_code from above list. lets try to handle that
    #X_train.zip_code = X_train.zip_code.astype(float).fillna(0.0)
    #X_test.zip_code = X_test.zip_code.astype(float).fillna(0.0)
    # could not convert string to float: 'N9A2H9'
    
    X_train.zip_code = pd.to_numeric(X_train.zip_code, errors="coerce").fillna(0.0)
    X_test.zip_code = pd.to_numeric(X_test.zip_code, errors="coerce").fillna(0.0)
    # Forces conversion (or set's to nan when errors="coerce") This will work even when astype will fail; 
    #its also series by series so it won't convert say a complete string column
    
    # drop string cols:
    X_train = X_train.drop(str_predictor_cols, axis=1)
    X_test = X_test.drop(str_predictor_cols, axis=1)
    
    #print(X_train.columns)
    
    #print(traindata.shape)
    #print('5: ', X_train.shape)
    #print(y_train.shape)
    
    
    # Count NaNs that the RF is not able to digest
    #print("Xtrain Nans: ", X_train.isnull().sum())
    #print("Xtest Nans: ", X_test.isnull().sum())
    
    # latlon contans a couple of nans... do something with them... F is not able to digest them
    # try fillna      # cannot drop for test
    X_train.fillna(axis=0, method="pad", inplace=True)  # use just prev value... kind of a hack -- just a few datapoints, shouldnt matter too much
    X_test.fillna(axis=0, method="pad", inplace=True)  # use just prev value... kind of a hack -- just a few datapoints, shouldnt matter too much
    
    # Count NaNs that the RF is not able to digest
    #print("Xtrain Nans: ", X_train.isnull().sum())
    #print("Xtest Nans: ", X_test.isnull().sum())
    # cannot drop for thest
    
    #print(traindata.head())
    #print(traindata.info())
    
    #print(X_train.head())
    #print("\n")
    #print(X_test.head())
    
    #print("\n")
    #print(X_train.columns)
    
    
    # lets apply RF
    # No scaling needed (decision trees...)
    
    # train test split done already
    
    from sklearn.ensemble import RandomForestClassifier
    
    clf = RandomForestClassifier()
    clf.fit(X_train, y_train)
    
    y_pred_proba = clf.predict_proba(X_test)[:,1]  # pred proba for positive class, ie compliance=1 will be paid
    #print("ypred shape: ", y_pred_paid.shape)  # ok length
    
    #print(X_test.index)
    
    y_pred_serie = pd.Series(data=y_pred_proba, index=X_test.index)
    
    return y_pred_serie# Your answer here



In [98]:
blight_model()
#res = blight_model()
#print(res)

/opt/conda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2827: DtypeWarning: Columns (11,12,31) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


    ticket_id                                     agency_name  \
0       22056  Buildings, Safety Engineering & Env Department   
1       27586  Buildings, Safety Engineering & Env Department   
5       22046  Buildings, Safety Engineering & Env Department   
6       18738  Buildings, Safety Engineering & Env Department   
7       18735  Buildings, Safety Engineering & Env Department   
8       18733  Buildings, Safety Engineering & Env Department   
9       28204  Buildings, Safety Engineering & Env Department   
12      18743  Buildings, Safety Engineering & Env Department   
13      18741  Buildings, Safety Engineering & Env Department   
14      18978  Buildings, Safety Engineering & Env Department   
18      18746  Buildings, Safety Engineering & Env Department   
19      18744  Buildings, Safety Engineering & Env Department   
21      26846  Buildings, Safety Engineering & Env Department   
22      26848  Buildings, Safety Engineering & Env Department   
23      28209  Buildings,

/opt/conda/lib/python3.5/site-packages/ipykernel/__main__.py:143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


ticket_id
284932    0.000000
285362    0.000000
285361    0.000000
285338    0.000000
285346    0.000000
285345    0.100000
285347    0.000000
285342    0.200000
285530    0.100000
284989    0.000000
285344    0.100000
285343    0.000000
285340    0.100000
285341    0.200000
285349    0.100000
285348    0.000000
284991    0.000000
285532    0.000000
285406    0.000000
285001    0.000000
285006    0.000000
285405    0.000000
285337    0.000000
285496    0.000000
285497    0.000000
285378    0.000000
285589    0.000000
285585    0.100000
285501    0.000000
285581    0.000000
            ...   
376367    0.000000
376366    0.000000
376362    0.016667
376363    0.200000
376365    0.000000
376364    0.100000
376228    0.800000
376265    0.000000
376286    0.533333
376320    0.000000
376314    0.000000
376327    0.700000
376385    0.500000
376435    0.300000
376370    0.300000
376434    0.100000
376459    0.000000
376478    0.000000
376473    0.100000
376484    0.100000
376482    0.000000
37

In [92]:
# Sandbox stuff

traindata = pd.read_csv("train.csv", encoding = 'ISO-8859-1')

traindata = traindata[(traindata["compliance"] == 0) | (traindata["compliance"] == 1)]
    # enclose in parantheses...
    
print(traindata.head(15))
    
    # predictor columns (that can be find both from train and test)
predictor_cols = ["ticket_id", "agency_name", "inspector_name", "violator_name", 
                    "violation_street_number", "violation_street_name", "violation_zip_code",
                    "mailing_address_str_number", "mailing_address_str_name", "city", "state", "zip_code",
                    "non_us_str_code", "country",
                    "ticket_issued_date", "hearing_date", 
                    "violation_code", "violation_description", "disposition",
                    "fine_amount", "admin_fee", "state_fee", "late_fee","discount_amount","clean_up_cost",
                    "judgment_amount", "grafitti_status"]
    


X_train = traindata[predictor_cols]

#X_train["grafitti_status"].value_counts()
#X_train["city"].value_counts()
#X_train["violation_zip_code"].value_counts()
X_train["zip_code"].value_counts()

/opt/conda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (11,12,31) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


    ticket_id                                     agency_name  \
0       22056  Buildings, Safety Engineering & Env Department   
1       27586  Buildings, Safety Engineering & Env Department   
5       22046  Buildings, Safety Engineering & Env Department   
6       18738  Buildings, Safety Engineering & Env Department   
7       18735  Buildings, Safety Engineering & Env Department   
8       18733  Buildings, Safety Engineering & Env Department   
9       28204  Buildings, Safety Engineering & Env Department   
12      18743  Buildings, Safety Engineering & Env Department   
13      18741  Buildings, Safety Engineering & Env Department   
14      18978  Buildings, Safety Engineering & Env Department   
18      18746  Buildings, Safety Engineering & Env Department   
19      18744  Buildings, Safety Engineering & Env Department   
21      26846  Buildings, Safety Engineering & Env Department   
22      26848  Buildings, Safety Engineering & Env Department   
23      28209  Buildings,

48221     4524
48227     4467
48235     4299
48219     3940
48228     3681
48224     3559
48238     3475
48205     2948
48227     2849
48204     2807
48221     2689
48234     2595
48235     2543
48228     2345
48223     2343
48209     2282
48206     2229
48213     2203
48203     2109
48075     2104
48210     2091
48238     1960
48219     1935
48224     1862
48205     1816
48207     1679
48202     1679
48076     1565
48204     1550
48234     1462
          ... 
91770        1
90620        1
74857        1
48305        1
9313         1
42080        1
91231        1
91217        1
91362        1
31321        1
75039        1
91401        1
98327        1
37217        1
48250        1
95206        1
92908        1
9512         1
30901        1
48714        1
98618        1
60601        1
75030        1
91405        1
8205         1
N8N3N1       1
3146         1
42221        1
42219        1
91748        1
Name: zip_code, dtype: int64